<div class="clearfix" style="padding: 10px; padding-left: 0px">
<img src="https://raw.githubusercontent.com/jupyter/nature-demo/master/images/jupyter-logo.png" width="100px" style="display: inline-block; margin-top: 10px;">
<a href="https://mybinder.org"><img src="https://mybinder.org/static/logo.svg" width="120px" class="pull-right" style="display: inline-block; margin: 0px;"></a>
</div>

# Linear Algebra in SciPy

This notebook is a part of [Lectures on scientific computing with Python](http://github.com/jrjohansson/scientific-python-lectures) by [J.R. Johansson](http://jrjohansson.github.io). 

<div class="alert alert-info" role="alert" style="margin: 10px">
<p>This notebook is running on a server that was **launched just for you**. Your changes will be reset once server restarts due to inactivity, so don't rely on it for anything you want to last.
</p>
</div>

Your server is hosted thanks to [binder](https://mybinder.org), service that allows you to create custom computing environments based on GitHub repositories.

In [1]:
from scipy import *

## Linear algebra module

The linear algebra module contains a lot of matrix related functions, including linear equation solving, eigenvalue solvers, matrix functions (for example matrix-exponentiation), a number of different decompositions (SVD, LU, cholesky), etc. 

Detailed documetation is available at: http://docs.scipy.org/doc/scipy/reference/linalg.html 

Here we will look at how to use some of these functions.

## Linear equation systems

Linear equation systems on the matrix form

$A x = b$

where $A$ is a matrix and $x,b$ are vectors can be solved like:

In [2]:
from scipy.linalg import *

In [3]:
A = array([[3, 2, 0], [1, -1, 0], [0, 5, 1]])
b = array([2, 4, -1])

In [4]:
x = solve(A, b)

x

array([ 2., -2.,  9.])

In [5]:
# check
dot(A, x) - b

array([ 0.,  0.,  0.])

We can also do the same with

$A X = B$

where $A, B, X$ are matrices:

In [6]:
A = rand(3,3)
B = rand(3,3)

In [7]:
X = solve(A, B)

In [8]:
X

array([[-1.40305951, -0.00944967, -0.71848024],
       [ 0.7899836 ,  0.00719044,  0.53697354],
       [ 0.99991573,  0.10033505,  0.65333977]])

In [9]:
# check
norm(dot(A, X) - B)

1.4703252574089423e-16

## Eigenvalues and eigenvectors

The eigenvalue problem for a matrix $A$:

$\displaystyle A v_n = \lambda_n v_n$

where $v_n$ is the $n$th eigenvector and $\lambda_n$ is the $n$th eigenvalue.

To calculate eigenvalues of a matrix, use the `eigvals` and for calculating both eigenvalues and eigenvectors, use the function `eig`:

In [10]:
evals = eigvals(A)

In [11]:
evals

array([ 1.49624589+0.j        , -0.19746277+0.36041789j,
       -0.19746277-0.36041789j])

In [12]:
evals, evecs = eig(A)

In [13]:
evals

array([ 1.49624589+0.j        , -0.19746277+0.36041789j,
       -0.19746277-0.36041789j])

In [14]:
evecs

array([[ 0.42706097+0.j        ,  0.69959774+0.j        ,  0.69959774-0.j        ],
       [ 0.30248844+0.j        , -0.34507806+0.35925613j,
        -0.34507806-0.35925613j],
       [ 0.85212656+0.j        , -0.18451756-0.4778833j ,
        -0.18451756+0.4778833j ]])

The eigenvectors corresponding to the $n$th eigenvalue (stored in `evals[n]`) is the $n$th *column* in `evecs`, i.e., `evecs[:,n]`. To verify this, let's try mutiplying eigenvectors with the matrix and compare to the product of the eigenvector and the eigenvalue:

In [15]:
n = 1

norm(dot(A, evecs[:,n]) - evals[n] * evecs[:,n])

3.206377205567385e-16

There are also more specialized eigensolvers, like the `eigh` for Hermitian matrices. 

## Matrix operations

In [16]:
# the matrix inverse
inv(A)

array([[-1.59001585, -1.80010496,  1.77082418],
       [ 1.56861916,  0.10221446, -0.58518279],
       [-0.0091804 ,  2.40892876, -0.1821828 ]])

In [17]:
# determinant
det(A)

0.2527048597795389

In [18]:
# norms of various orders
norm(A, ord=2), norm(A, ord=Inf)

(1.8370185038057469, 2.5997131967873406)

## Sparse matrices

Sparse matrices are often useful in numerical simulations dealing with large systems, if the problem can be described in matrix form where the matrices or vectors mostly contains zeros. Scipy has a good support for sparse matrices, with basic linear algebra operations (such as equation solving, eigenvalue calculations, etc).

There are many possible strategies for storing sparse matrices in an efficient way. Some of the most common are the so-called coordinate form (COO), list of list (LIL) form,  and compressed-sparse column CSC (and row, CSR). Each format has some advantanges and disadvantages. Most computational algorithms (equation solving, matrix-matrix multiplication, etc) can be efficiently implemented using CSR or CSC formats, but they are not so intuitive and not so easy to initialize. So often a sparse matrix is initially created in COO or LIL format (where we can efficiently add elements to the sparse matrix data), and then converted to CSC or CSR before used in real calcalations.

For more information about these sparse formats, see e.g. http://en.wikipedia.org/wiki/Sparse_matrix

When we create a sparse matrix we have to choose which format it should be stored in. For example, 

In [19]:
from scipy.sparse import *

In [20]:
# dense matrix
M = array([[1,0,0,0], [0,3,0,0], [0,1,1,0], [1,0,0,1]]); M

array([[1, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 0, 1]])

In [21]:
# convert from dense to sparse
A = csr_matrix(M); A

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [22]:
# convert from sparse to dense
A.todense()

matrix([[1, 0, 0, 0],
        [0, 3, 0, 0],
        [0, 1, 1, 0],
        [1, 0, 0, 1]], dtype=int64)

More efficient way to create sparse matrices: create an empty matrix and populate with using matrix indexing (avoids creating a potentially large dense matrix)

In [23]:
A = lil_matrix((4,4)) # empty 4x4 sparse matrix
A[0,0] = 1
A[1,1] = 3
A[2,2] = A[2,1] = 1
A[3,3] = A[3,0] = 1
A

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in LInked List format>

In [24]:
A.todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  1.]])

Converting between different sparse matrix formats:

In [25]:
A

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in LInked List format>

In [26]:
A = csr_matrix(A); A

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [27]:
A = csc_matrix(A); A

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Column format>

We can compute with sparse matrices like with dense matrices:

In [28]:
A.todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  1.]])

In [29]:
(A * A).todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  9.,  0.,  0.],
        [ 0.,  4.,  1.,  0.],
        [ 2.,  0.,  0.,  1.]])

In [30]:
A.todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  1.]])

In [31]:
A.dot(A).todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  9.,  0.,  0.],
        [ 0.,  4.,  1.,  0.],
        [ 2.,  0.,  0.,  1.]])

In [32]:
v = array([1,2,3,4])[:,newaxis]; v

array([[1],
       [2],
       [3],
       [4]])

In [33]:
# sparse matrix - dense vector multiplication
A * v

array([[ 1.],
       [ 6.],
       [ 5.],
       [ 5.]])

In [34]:
# same result with dense matrix - dense vector multiplcation
A.todense() * v

matrix([[ 1.],
        [ 6.],
        [ 5.],
        [ 5.]])

## Further reading

* Check out more introductory notebooks in **Juno**!
* http://www.scipy.org - The official web page for the SciPy project.
* http://docs.scipy.org/doc/scipy/reference/tutorial/index.html - A tutorial on how to get started using SciPy. 
* https://github.com/scipy/scipy/ - The SciPy source code. 

## Versions

In [35]:
%reload_ext version_information

%version_information numpy, matplotlib, scipy

Software versions
Python 3.6.1 64bit [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
IPython 6.2.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.12.1
matplotlib 2.0.2
scipy 0.19.0
Mon Apr 30 20:07:04 2018 BST